# Лабораторная работа №3

# Двойственный симплекс метод

### Выполнил: Яковлев Артур, гр. 853501

### Проверила: Костюкова О.И.

#### Вариант 28

Импортируем все необходимые модули

In [1]:
import numpy as np
import scipy.linalg as sla

Исходные данные для тестового примера:

In [27]:
test_dataset = {
    'A': np.array([[2., -3.,  4., 0., 2.,  0., -2., 7.],
                   [0.,  2., -2., 2., 1., -1.,  4., 3.],
                   [0.,  4.,  1., 1., 3.,  5.,  1., 2.]]),
    'b': np.array([0., -4., -18.]),
    'c': np.array([-2., 15., -11., 8., 3., -2., 13., 7.]),
    'y': np.array([-1., 4., 1.]),
    'jb': [0, 1, 2],
}

Реализуем итерацию нашего алгоритма. Данная функция принимает в себя все исходные данные, обратную матрицу $B$ и вектор $\delta$. Итерация состоит из 9 шагов, каждый из которых приведен в коде ниже.

In [35]:
def iteration(dataset, B, delta):  # B is a np.array, delta is Dict[int, float]
    
    # Step 1
    x_b = B.dot(dataset['b'])
    print('A: ', dataset['A'])
    print('B: ', B)
    print(f'Current plan: {x_b}')
    
    # Step 2
    if np.all(x_b > -1e-9):
        return None, x_b
    
    # Step 3
    k = np.argmin(x_b)
    print(f'k: {k}')
    jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['jb']]
    print(f'jn: {jn}')
    mu = {j: B[k, :].dot(dataset['A'][:, j]) for j in jn}
    print(f'mu: {mu}')
    
    # Step 4
    sigma = {j: -delta[j] / mu[j] if mu[j] < 1e-9 else np.inf for j in jn}
    print(f'sigma: {sigma}')
    
    # Step 5, 6
    sigma0 = min(sigma, key=sigma.get)
    if sigma[sigma0] == np.inf:
        return None, None
    
    # Step 7
    idx = dataset['jb'][k]
    del dataset['jb'][k]
    dataset['jb'].append(sigma0)
    dataset['jb'].sort()
    print(f'New jb: {dataset["jb"]}')
    
    # Step 8
    delta.pop(sigma0)
    for key in delta:
        delta[key] += sigma[sigma0] * mu[key]
    delta[idx] = sigma[sigma0]
    print(f'New delta: {delta}')
    
    #Step 9
    B = sla.inv(dataset['A'][:, dataset['jb']])
    return B, delta

Далее реализуем сам алгоритм решения. Функция принимает в себя набор исходных данных, состоящих из матрицы $A$, векторов $b$, $c$, $y$ и массива индексов $J_b$, а также максимальное число итераций алгоритма (по умолчанию 20). В основном цикле программы с помощью функции итерации обновляются матрица $B$ и вектор $\delta$ и осуществляется вывод оптимального решения в случае, если оно было получено.

In [36]:
def solve(dataset, max_iter=20):
    B = sla.inv(dataset['A'][:, dataset['jb']])
    jn = [i for i in range(dataset['A'].shape[1]) if i not in dataset['jb']]
    init_delta = np.dot(dataset['c'][dataset['jb']], B @ dataset['A'][:, jn]) - dataset['c'][jn]
    delta = {jn[i]: init_delta[i] for i in range(len(jn))}
    for i in range(max_iter):
        print('\n==============================================\n')
        print(f'Iteration {i + 1}\n')
        B, delta = iteration(dataset, B, delta)
        if B is None:
            if delta is None:
                print('\nNo solution\n')
            else:
                plan = np.zeros(dataset['c'].shape)
                plan[dataset['jb']] = delta
                print(f'\nOptimized plan: {plan}')
                print(f"cx: {dataset['c'][dataset['jb']].dot(delta)}")
            return
    print(f'\nBest found plan: {delta}')

Проверим наш алгоритм на тестовом наборе данных

In [37]:
solve(test_dataset)



Iteration 1

A:  [[ 2. -3.  4.  0.  2.  0. -2.  7.]
 [ 0.  2. -2.  2.  1. -1.  4.  3.]
 [ 0.  4.  1.  1.  3.  5.  1.  2.]]
B:  [[ 0.5   0.95 -0.1 ]
 [ 0.    0.1   0.2 ]
 [ 0.   -0.4   0.2 ]]
Current plan: [-2. -4. -2.]
k: 1
jn: [3, 4, 5, 6, 7]
mu: {3: 0.4, 4: 0.7000000000000001, 5: 0.9, 6: 0.6000000000000001, 7: 0.7000000000000001}
sigma: {3: inf, 4: inf, 5: inf, 6: inf, 7: inf}

No solution



В тестовом примере получили, что решения нет, т. к. все значения в векторе $\sigma$ уходят в бесконечность

Далее рассмотрим задачу для варианта 28. Исходные данные:

In [38]:
var28 = {
    'A': np.array([[ 1.,  2., -1.,  0.,  3.,  2.,  3.,  1.,  0.],
                   [ 1.,  0.,  0.,  1., -1.,  3.,  0.,  7.,  5.],
                   [ 0.,  1., -4.,  1., -3.,  1.,  0.,  1.,  0.],
                   [ 1.,  2., -3.,  1., -1.,  2.,  5.,  0.,  4.]]),
    'b': np.array([2., 3., -3., 3.]),
    'c': np.array([8., 8., 2., 0., 21., 12., 16., 16., 9.]),
    'y': np.array([5., 2., -3., 1.]),
    'jb': [0, 3, 4, 7],
}

In [39]:
solve(var28)



Iteration 1

A:  [[ 1.  2. -1.  0.  3.  2.  3.  1.  0.]
 [ 1.  0.  0.  1. -1.  3.  0.  7.  5.]
 [ 0.  1. -4.  1. -3.  1.  0.  1.  0.]
 [ 1.  2. -3.  1. -1.  2.  5.  0.  4.]]
B:  [[-2.          0.71428571 -3.          2.28571429]
 [ 3.         -1.          4.         -2.        ]
 [ 1.         -0.28571429  1.         -0.71428571]
 [ 0.          0.14285714  0.         -0.14285714]]
Current plan: [ 14. -15.  -4.   0.]
k: 1
jn: [1, 2, 5, 6, 8]
mu: {1: 6.0, 2: -13.0, 5: 3.0, 6: -1.0, 8: -13.0}
sigma: {1: inf, 2: 0.15384615384615324, 5: inf, 6: 4.000000000000007, 8: 0.3846153846153854}
New jb: [0, 2, 4, 7]
New delta: {1: 1.9230769230769211, 5: 3.461538461538453, 6: 3.846153846153854, 8: 3.0000000000000187, 3: 0.15384615384615324}


Iteration 2

A:  [[ 1.  2. -1.  0.  3.  2.  3.  1.  0.]
 [ 1.  0.  0.  1. -1.  3.  0.  7.  5.]
 [ 0.  1. -4.  1. -3.  1.  0.  1.  0.]
 [ 1.  2. -3.  1. -1.  2.  5.  0.  4.]]
B:  [[-0.35164835  0.16483516 -0.8021978   1.18681319]
 [-0.23076923  0.07692308 -0.3076

Аналогично решим задачу для варианта 29.

In [40]:
var29 = {
    'A': np.array([
        [1., 2., -1.,  0.,  3.,  2., 3.,  1., 5.],
        [1., 0.,  0., -1., -1., -3., 0., -6., 1.],
        [0., 1., -4.,  1.,  3.,  1., 0.,  1., 0.],
        [1., 2., -3.,  1., -1.,  2., 5.,  0., 4.]
    ]),
    'b': np.array([12., -2., 8., 12.]),
    'c': np.array([-9., -13., 19., -5., 2., -7., -32., 11., -28.]),
    'y': np.array([-1., -3., -1., -5.]),
    'jb': [0, 1, 4, 8],
}

In [41]:
solve(var29)



Iteration 1

A:  [[ 1.  2. -1.  0.  3.  2.  3.  1.  5.]
 [ 1.  0.  0. -1. -1. -3.  0. -6.  1.]
 [ 0.  1. -4.  1.  3.  1.  0.  1.  0.]
 [ 1.  2. -3.  1. -1.  2.  5.  0.  4.]]
B:  [[-0.16666667  1.27777778  0.55555556 -0.11111111]
 [-0.5        -0.16666667  0.66666667  0.66666667]
 [ 0.16666667  0.05555556  0.11111111 -0.22222222]
 [ 0.33333333 -0.22222222 -0.44444444 -0.11111111]]
Current plan: [-1.44444444  7.66666667  0.11111111 -0.44444444]
k: 0
jn: [2, 3, 5, 6, 7]
mu: {2: -1.7222222222222225, 3: -0.8333333333333333, 5: -3.833333333333333, 6: -1.0555555555555551, 7: -7.277777777777777}
sigma: {2: 0.5806451612903204, 3: 2.399999999999998, 5: 0.782608695652174, 6: 3.7894736842105243, 7: 0.6870229007633565}
New jb: [1, 2, 4, 8]
New delta: {3: 1.5161290322580645, 5: 0.7741935483871054, 6: 3.387096774193547, 7: 0.7741935483870952, 0: 0.5806451612903204}


Iteration 2

A:  [[ 1.  2. -1.  0.  3.  2.  3.  1.  5.]
 [ 1.  0.  0. -1. -1. -3.  0. -6.  1.]
 [ 0.  1. -4.  1.  3.  1.  0.  1.  0.]